In [1]:
import numpy as np

from scipy import linalg

import Ipynb_importer

import conductor_calculation as cc

import new_topology as tp



from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt
font_set = FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=14)
class ChainNetwork:

    def __init__(self,

                 name="",

                 h=1,  # 基波、谐波次数。h = 1,基波；h >1 ,谐波

                 delta_length=0.5,  # 基本分段长度（km）

                 m=6,  # 归并后的导线数                       ((需要配合impel——ui改动））

                 topology=tp.Topology(),  # 供电网络拓扑结构数据 ((改变数据库时改动））

                 conductor_calculation=cc.conductor_calculation(),

                 ):

        self.name = name

        self.topology = topology

        self.conductor_calculation = conductor_calculation

        # print(len(self.topology.locomotive))

        self.h = h

        self.m = m

        self.section_length = self.topology.section_length  # 供电臂总长度

        self.delta_length = delta_length  # 基本分段长度

        self.n = int(np.round(self.section_length / self.delta_length) + 1)  # 分割面数，即网络节点数

        self.distances = self.__set_distances()  # 分割点位置距离

        self.segment_lengths = self.__set_segment_lengths()  # 各分割段长度

        self.z = np.zeros((self.n - 1, self.m, self.m), np.complex128)  # 网络纵向阻抗矩阵

        self.y = np.zeros((self.n, self.m, self.m), np.complex128)  # 网络横向导纳矩阵

        self.Un = np.zeros((self.n, self.m, 1), np.complex128)  # 节点电压向量

        self.In = np.zeros((self.n - 1, self.m, 1), np.complex128)  # 纵向导线电流向量

        self.Gn = np.zeros((self.m * self.n, 1), np.complex128)

    def reset(self, h, m, delta_length, dbfile="calculate_system.db"):

        # self.topology = topology

        self.topology.reset_topology()

        self.topology.set_topology(dbfile)####((更换数据库时要改))

        self.conductor_calculation.set_data(dbfile)

        self.h = h

        self.m = m

        self.delta_length = delta_length

        self.dbfile = dbfile

        self.section_length = self.topology.section_length

        self.name = self.topology.name

        self.n = int(np.round(self.section_length / self.delta_length) + 1)  # 分割面数，即网络节点

        # print(self.n)


        self.distances = self.__set_distances()  # 分割点位置距离

        self.segment_lengths = self.__set_segment_lengths()  # 各分割段长度

        self.z = np.zeros((self.n - 1, self.m, self.m), np.complex128)  # 网络纵向阻抗矩阵

        self.y = np.zeros((self.n, self.m, self.m), np.complex128)  # 网络横向导纳矩阵

        self.Un = np.zeros((self.n, self.m, 1), np.complex128)  # 节点电压向量

        self.In = np.zeros((self.n - 1, self.m, 1), np.complex128)  # 纵向导线电流向量

        self.Gn = np.zeros((self.m * self.n, 1), np.complex128)

    def __set_distances(self):

        distances = np.arange(0, self.n, 1) * self.delta_length

        distances[self.n - 1] = self.section_length

        # print(distances)
        #
        # print(len(distances))

        return distances

    def __set_segment_lengths(self):

        segment_lengths = np.zeros(self.n - 1, np.float64)

        segment_lengths[:self.n - 1] = self.distances[1:self.n] - self.distances[0:self.n - 1]  # 分割段长度 （km）

        return segment_lengths

    def set_z_y(self):

        unit_z = self.__calc_unit_z()

        unit_yc = self.__calc_unit_yc()

        for i in range(self.n - 1):
            self.z[i, :, :] = unit_z * self.segment_lengths[i]

        for i in range(1, self.n - 1):
            self.y[i, :, :] = unit_yc * (self.segment_lengths[i - 1] / 2 + self.segment_lengths[i] / 2)

            self.y[0, :, :] = unit_yc * (self.segment_lengths[0] / 2)

            self.y[self.n - 1, :, :] = unit_yc * (self.segment_lengths[self.n - 2] / 2)


    def construct_M(self):

        n = self.n

        m = self.m

        Dk = np.zeros((n, m, m), np.complex128)

        Mk = np.zeros((n, m, m), np.complex128)

        M = np.zeros((m * n, m * n), np.complex128)

        for k in range(1, n):  # 生产Dk元素矩阵

            Dk[k, :, :] = -linalg.inv(self.z[k - 1, :, :])

        for k in range(1, n - 1):  # 生成Mk元素矩阵

            Mk[k, :, :] = self.y[k, :, :] + linalg.inv(self.z[k, :, :]) + linalg.inv(self.z[k - 1, :, :])

        Mk[0, :, :] = self.y[0, :, :] + linalg.inv(self.z[0, :, :])

        Mk[n - 1, :, :] = self.y[n - 1, :, :] + linalg.inv(self.z[n - 2, :, :])

        for i in range(n):

            if i == 0:

                M[0:m, 0:m] = Mk[0, :, :]

            else:

                k = i * m  # 分块起点号

                M[k:k + m, k - m:k] = Dk[i, :, :]

                M[k - m: k, k:k + m] = Dk[i, :, :]

                M[k:k + m, k:k + m] = Mk[i, :, :]

        return M

    def solution(self):

        M = self.construct_M()

        W = linalg.inv(M)  ##改动

        U = np.dot(W, self.Gn)

        Un = np.zeros((self.m, self.n), np.complex128)

        for i in range(self.n):
            k = int(i) * self.m

            j = int(i + 1) * self.m


            Un[:, i] = U[k:j, 0]

        In = np.zeros((self.m, self.n - 1), np.complex128)  #

        for k in range(self.n - 1):

            z = self.z[k, :, :]

            z = linalg.inv(z)

            u = Un[:, k] - Un[:, k + 1]

            u = np.mat(u)

            u = u.T

            Ini = np.dot(z, u)

            Ini = np.mat(Ini)

            Ini = Ini.T

            for j in range(0, self.m, 1):
                In[j:, k] = Ini[0, j]


        return Un, In

    def add_y(self):

        if self.m == 6:

            y_to_all = cc.set_connection_matrix(self.m, 0, 3) + cc.set_connection_matrix(self.m, 1,
                                                                                         4) + cc.set_connection_matrix(
                self.m, 2, 5)  # 上下行全并联导纳连接矩阵

            y_e1_g = cc.set_connction_matrix_g(self.m, 2)  # 综合地线e1连接大地g ，导纳关联矩阵

            y_e2_g = cc.set_connction_matrix_g(self.m, 5)  # 综合地线e2连接大地g ，导纳关联矩

            y_ra1_g = cc.set_connction_matrix_g(self.m, 2)  # 钢轨ra1连接大地g, 导纳关联矩阵

            y_ra3_g = cc.set_connction_matrix_g(self.m, 5)  # 钢轨ra3连接大地g, 导纳关联矩阵

            cross_connections = self.topology.cross_connections[0]

            y0 = 10 ** 8

            for x in cross_connections.to_all:  # 全并联连接线

                k = int(np.round(x / self.delta_length))

                self.y[k, :, :] = self.y[k, :, :] + y_to_all * y0

            for x in cross_connections.ra1_g:

                k = int(np.round(x / self.delta_length))

                if k < self.n:
                     rg = cross_connections.ra1_g.get(x)

                    # print('rg')
                    # print(rg)

                     self.y[k, :, :] = self.y[k, :, :] + y_ra1_g * (1 / rg)

            for x in cross_connections.ra3_g:

                k = int(np.round(x / self.delta_length))

                if k < self.n:
                    rg = cross_connections.ra3_g.get(x)

                    self.y[k, :, :] = self.y[k, :, :] + y_ra3_g * (1 / rg)

            for x in cross_connections.e1_g:

                k = int(np.round(x / self.delta_length))

                if k < self.n:
                    rg = cross_connections.e1_g.get(x)

                    self.y[k, :, :] = self.y[k, :, :] + y_e1_g * (1 / rg)

            for x in cross_connections.e2_g:

                k = int(np.round(x / self.delta_length))

                if k < self.n:
                    rg = cross_connections.e2_g.get(x)

                    self.y[k, :, :] = self.y[k, :, :] + y_e2_g * (1 / rg)

        elif self.m == 10:

            y_to_all = cc.set_connection_matrix(self.m, 0, 5) + cc.set_connection_matrix(self.m, 1,
                                                                                         6) + cc.set_connection_matrix(
                self.m, 2, 7)  # 上下行全并联导纳连接矩阵

            y_e1_g = cc.set_connction_matrix_g(self.m, 4)  # 综合地线e1连接大地g ，导纳关联矩阵

            y_e2_g = cc.set_connction_matrix_g(self.m, 9)  # 综合地线e2连接大地g ，导纳关联矩

            y_ra1_g = cc.set_connction_matrix_g(self.m, 4)  # 钢轨ra1连接大地g, 导纳关联矩阵

            y_ra3_g = cc.set_connction_matrix_g(self.m, 9)  # 钢轨ra3连接大地g, 导纳关联矩阵

            y_e1_ra1 = cc.set_connection_matrix(self.m, 2, 4)  # 钢轨ra1连接综合地线e1,导纳关联矩阵

            y_e2_ra3 = cc.set_connection_matrix(self.m, 7, 9)  # 钢轨ra2连接综合地线e2,导纳关联矩阵

            cross_connections = self.topology.cross_connections[0]

            y0 = 10 ** 8  # 连接导线的电导率

            for x in cross_connections.to_all:  # 全并联连接线

                k = int(np.round(x / self.delta_length))

                self.y[k, :, :] = self.y[k, :, :] + y_to_all * y0

            for x in cross_connections.ra1_g:

                k = int(np.round(x / self.delta_length))

                if k < self.n:
                    rg = cross_connections.ra1_g.get(x)

                    self.y[k, :, :] = self.y[k, :, :] + y_ra1_g * (1 / rg)

            for x in cross_connections.ra3_g:

                k = int(np.round(x / self.delta_length))

                if k < self.n:
                    rg = cross_connections.ra3_g.get(x)

                    self.y[k, :, :] = self.y[k, :, :] + y_ra3_g * (1 / rg)

            for x in cross_connections.e1_g:

                k = int(np.round(x / self.delta_length))

                if k < self.n:
                    rg = cross_connections.e1_g.get(x)

                    self.y[k, :, :] = self.y[k, :, :] + y_e1_g * (1 / rg)

            for x in cross_connections.e2_g:

                k = int(np.round(x / self.delta_length))

                if k < self.n:
                    rg = cross_connections.e2_g.get(x)

                    self.y[k, :, :] = self.y[k, :, :] + y_e2_g * (1 / rg)

            for x in cross_connections.e1_ra1:

                k = int(np.round(x / self.delta_length))

                if k < self.n:
                    self.y[k, :, :] = self.y[k, :, :] + y_e1_ra1 * y0

            for x in cross_connections.e2_ra3:

                k = int(np.round(x / self.delta_length))

                if k < self.n:
                    self.y[k, :, :] = self.y[k, :, :] + y_e2_ra3 * y0

        elif self.m == 4:

            cross_connections = self.topology.cross_connections[0]

            y_e1_g = cc.set_connction_matrix_g(4, 1)  # 综合地线e1连接大地g ，导纳关联矩阵

            y_e2_g = cc.set_connction_matrix_g(4, 3)  # 综合地线e2连接大地g ，导纳关联矩

            y_ra1_g = cc.set_connction_matrix_g(4, 1)  # 钢轨ra1连接大地g, 导纳关联矩阵

            y_ra3_g = cc.set_connction_matrix_g(4, 3)  # 钢轨ra3连接大地g, 导纳关联矩阵


            for x in cross_connections.ra1_g:

                k = int(np.round(x / self.delta_length))

                if k < self.n:
                    rg = cross_connections.ra1_g.get(x)

                    self.y[k, :, :] = self.y[k, :, :] + y_ra1_g * (1 / rg)

            for x in cross_connections.ra3_g:

                k = int(np.round(x / self.delta_length))

                if k < self.n:
                    rg = cross_connections.ra3_g.get(x)

                    self.y[k, :, :] = self.y[k, :, :] + y_ra3_g * (1 / rg)

            for x in cross_connections.e1_g:

                k = int(np.round(x / self.delta_length))

                if k < self.n:
                    rg = cross_connections.e1_g.get(x)

                    self.y[k, :, :] = self.y[k, :, :] + y_e1_g * (1 / rg)

            for x in cross_connections.e2_g:

                k = int(np.round(x / self.delta_length))

                if k < self.n:
                    rg = cross_connections.e2_g.get(x)

                    self.y[k, :, :] = self.y[k, :, :] + y_e2_g * (1 / rg)

    def add_y_source(self):

        if self.m == 6:

            traction = self.topology.traction_transformer

            self.y[0, 0, 0] = (1 / ((((110 / traction[0].transformer_ratio)**2/traction[0].nominal_capacity)*((traction[0].short_circuit_voltage_percentage)/100)) * self.h)) + self.y[0, 0, 0]  # 牵引变压器内阻抗    traction[0].zs

            self.y[0, 1, 0] = - (1 / ((((110 / traction[0].transformer_ratio)**2/traction[0].nominal_capacity)*((traction[0].short_circuit_voltage_percentage)/100)) * self.h)) + self.y[0, 1, 0]

            self.y[0, 0, 1] = - (1 / ((((110 / traction[0].transformer_ratio)**2/traction[0].nominal_capacity)*((traction[0].short_circuit_voltage_percentage)/100)) * self.h)) + self.y[0, 0, 1]

            self.y[0, 1, 1] = (1 / ((((110 / traction[0].transformer_ratio)**2/traction[0].nominal_capacity)*((traction[0].short_circuit_voltage_percentage)/100)) * self.h)) + self.y[0, 1, 1]

            self.y[0, 0, 0] = self.y[0, 0, 0] + 10 ** 8

            self.y[0, 3, 0] = self.y[0, 3, 0] - 10 ** 8

            self.y[0, 0, 3] = self.y[0, 0, 3] - 10 ** 8

            self.y[0, 3, 3] = self.y[0, 3, 3] + 10 ** 8

            self.y[0, 1, 1] = self.y[0, 1, 1] + 10 ** 8

            self.y[0, 1, 4] = self.y[0, 1, 4] - 10 ** 8

            self.y[0, 4, 1] = self.y[0, 4, 1] - 10 ** 8

            self.y[0, 4, 4] = self.y[0, 4, 4] + 10 ** 8


        elif self.m == 4:

            traction = self.topology.traction_transformer

            self.y[0, 0, 0] = (1 / ((((110 / traction[0].transformer_ratio)**2/traction[0].nominal_capacity)*((traction[0].short_circuit_voltage_percentage)/100)) * self.h)) + self.y[0, 0, 0]  # 牵引变压器内阻抗    traction[0].zs

            self.y[0, 1, 0] = - (1 / ((((110 / traction[0].transformer_ratio)**2/traction[0].nominal_capacity)*((traction[0].short_circuit_voltage_percentage)/100)) * self.h)) + self.y[0, 1, 0]

            self.y[0, 0, 1] = - (1 / ((((110 / traction[0].transformer_ratio)**2/traction[0].nominal_capacity)*((traction[0].short_circuit_voltage_percentage)/100)) * self.h)) + self.y[0, 0, 1]

            self.y[0, 1, 1] = (1 / ((((110 / traction[0].transformer_ratio)**2/traction[0].nominal_capacity)*((traction[0].short_circuit_voltage_percentage)/100)) * self.h)) + self.y[0, 1, 1]

            self.y[0, 0, 0] = self.y[0, 0, 0] + 10 ** 8

            self.y[0, 2, 0] = self.y[0, 2, 0] - 10 ** 8

            self.y[0, 0, 2] = self.y[0, 0, 2] - 10 ** 8

            self.y[0, 2, 2] = self.y[0, 2, 2] + 10 ** 8

            self.y[0, 1, 1] = self.y[0, 1, 1] + 10 ** 8

            self.y[0, 1, 3] = self.y[0, 1, 3] - 10 ** 8

            self.y[0, 3, 1] = self.y[0, 3, 1] - 10 ** 8

            self.y[0, 3, 3] = self.y[0, 3, 3] + 10 ** 8


        elif self.m == 10:

            traction = self.topology.traction_transformer

            self.y[0, 0, 0] = (1 / ((((110 / traction[0].transformer_ratio)**2/traction[0].nominal_capacity)*((traction[0].short_circuit_voltage_percentage)/100)) * self.h)) + self.y[0, 0, 0]  # 牵引变压器内阻抗    traction[0].zs

            self.y[0, 1, 0] = - (1 / ((((110 / traction[0].transformer_ratio)**2/traction[0].nominal_capacity)*((traction[0].short_circuit_voltage_percentage)/100)) * self.h)) + self.y[0, 1, 0]

            self.y[0, 0, 1] = - (1 / ((((110 / traction[0].transformer_ratio)**2/traction[0].nominal_capacity)*((traction[0].short_circuit_voltage_percentage)/100)) * self.h)) + self.y[0, 0, 1]

            self.y[0, 1, 1] = (1 / ((((110 / traction[0].transformer_ratio)**2/traction[0].nominal_capacity)*((traction[0].short_circuit_voltage_percentage)/100)) * self.h)) + self.y[0, 1, 1]

            self.y[0, 0, 0] = self.y[0, 0, 0] + 10 ** 8

            self.y[0, 5, 0] = self.y[0, 5, 0] - 10 ** 8

            self.y[0, 0, 5] = self.y[0, 0, 5] - 10 ** 8

            self.y[0, 5, 5] = self.y[0, 5, 5] + 10 ** 8

            self.y[0, 1, 1] = self.y[0, 1, 1] + 10 ** 8

            self.y[0, 1, 6] = self.y[0, 1, 6] - 10 ** 8

            self.y[0, 6, 1] = self.y[0, 6, 1] - 10 ** 8

            self.y[0, 6, 6] = self.y[0, 6, 6] + 10 ** 8


        return self.y

    # def __get_line_parameter(self):  # 获取导线参数
    #
    #     c_xy = []
    #
    #     c_resistance = []
    #
    #     c_mu_r = []
    #
    #     c_radius = []
    #
    #     c_equivalent_radius = []
    #
    #     c_rho = []
    #
    #     c_rd = []
    #
    #     earth_rou = self.topology.earth_rou
    #
    #     for line in self.topology.lines:
    #         c_xy.append([line.coordinater_x, line.coordinater_y])
    #
    #         c_resistance.append(line.resistance)
    #
    #         c_mu_r.append(line.mu_r)
    #
    #         c_radius.append(line.radius)
    #
    #         c_equivalent_radius.append(line.equivalent_radius)
    #
    #         c_rho.append(line.rho)
    #
    #         c_rd.append(line.resistance)
    #
    #     return c_xy, earth_rou, c_resistance, c_mu_r, c_radius, c_equivalent_radius, c_rho, c_rd

    def __get_line_parameter_test(self):  # 设定测试导线参数

        self.conductor_calculation.set_data(self.dbfile)

        c_xy = self.conductor_calculation.conductors_coordinator

        c_resistance = self.conductor_calculation.Rd

        c_mu_r = self.conductor_calculation.mu_r

        c_radius = self.conductor_calculation.conductors_calc_radius

        c_equivalent_radius = self.conductor_calculation.conductors_equivalent_radius

        c_rho = self.conductor_calculation.rho

        c_rd = self.conductor_calculation.Rd

        earth_rou = self.topology.earth_rou

        return c_xy, earth_rou, c_resistance, c_mu_r, c_radius, c_equivalent_radius, c_rho, c_rd

    def __calc_unit_z(self):

        """

        计算平行导线原始单位阻抗矩阵

        :return:     阻抗矩阵

        """

        c_xy, earth_rou, c_resistance, c_mu_r, c_radius, c_equivalent_radius, c_rho, c_rd = self.__get_line_parameter_test()

        f = 50 * self.h  # 频率

        z = cc.calc_Zf(f, c_xy, c_radius, c_resistance, c_rho, c_mu_r, earth_rou)
        # z = cc.calc_z(f, c_xy, c_equivalent_radius, c_rd, earth_rou)

        if self.m == 6:
            z = cc.merge_z(z, 0, 1)

            z = cc.merge_z(z, 2, 3)

            z = cc.merge_z(z, 2, 3)

            z = cc.merge_z(z, 2, 3)

            z = cc.merge_z(z, 3, 4)

            z = cc.merge_z(z, 5, 6)

            z = cc.merge_z(z, 5, 6)

            z = cc.merge_z(z, 5, 6)

        if self.m == 10:
            z = cc.merge_z(z, 0, 1)

            z = cc.merge_z(z, 2, 3)

            z = cc.merge_z(z, 5, 6)

            z = cc.merge_z(z, 7, 8)

        if self.m == 4:
            z = cc.merge_z(z, 0, 1)

            z = cc.merge_z(z, 1, 2)

            z = cc.merge_z(z, 1, 2)

            z = cc.merge_z(z, 2, 3)

            z = cc.merge_z(z, 3, 4)

            z = cc.merge_z(z, 3, 4)

        unit_z = z

        return unit_z

    def __calc_unit_yc(self):

        """

        计算导线的单位电容导纳矩阵

        :return:

        """

        c_xy, earth_rou, c_resistance, c_mu_r, c_radius, c_equivalent_radius, c_rho, c_rd = self.__get_line_parameter_test()

        p = cc.calc_potential_coefficient(c_xy, c_radius)

        if self.m == 6:
            p = cc.merge_potential_coefficient(p, 0, 1)  # 合并成6根导线

            p = cc.merge_potential_coefficient(p, 2, 3)

            p = cc.merge_potential_coefficient(p, 2, 3)

            p = cc.merge_potential_coefficient(p, 2, 3)

            p = cc.merge_potential_coefficient(p, 3, 4)

            p = cc.merge_potential_coefficient(p, 5, 6)

            p = cc.merge_potential_coefficient(p, 5, 6)

            p = cc.merge_potential_coefficient(p, 5, 6)

        if self.m == 10:

            p = cc.merge_potential_coefficient(p, 0, 1)  # 合并成6根导线

            p = cc.merge_potential_coefficient(p, 2, 3)

            p = cc.merge_potential_coefficient(p, 5, 6)

            p = cc.merge_potential_coefficient(p, 7, 8)


        if self.m == 4:

            p = cc.merge_potential_coefficient(p, 0, 1)  # 合并成4根导线

            p = cc.merge_potential_coefficient(p, 1, 2)

            p = cc.merge_potential_coefficient(p, 1, 2)

            p = cc.merge_potential_coefficient(p, 2, 3)

            p = cc.merge_potential_coefficient(p, 3, 4)

            p = cc.merge_potential_coefficient(p, 3, 4)


        b = cc.calc_B(p)  # 计算电容

        f = 50 * self.h

        unit_yc = 1j * 2 * np.pi * f * b  # 计算电容导纳

        return unit_yc

    def source(self):

        ##电源
        if self.m == 6 or self.m == 10:

            traction = self.topology.traction_transformer

            # print((((110 / traction[0].transformer_ratio)**2/traction[0].nominal_capacity)*((traction[0].short_circuit_voltage_percentage)/100)))

            self.Gn[0][0] = 55000 / ((((110 / traction[0].transformer_ratio)**2/traction[0].nominal_capacity)*((traction[0].short_circuit_voltage_percentage)/100)) * self.h)

            self.Gn[1][0] = -55000 / ((((110 / traction[0].transformer_ratio)**2/traction[0].nominal_capacity)*((traction[0].short_circuit_voltage_percentage)/100)) * self.h)

        elif self.m == 4:

            traction = self.topology.traction_transformer

            # print((((110 / traction[0].transformer_ratio) ** 2 / traction[0].nominal_capacity) * ((traction[0].short_circuit_voltage_percentage) / 100)))

            self.Gn[0][0] = 27500 / ((((110 / traction[0].transformer_ratio)**2/traction[0].nominal_capacity)*((traction[0].short_circuit_voltage_percentage)/100)) * self.h)

            self.Gn[1][0] = -27500 / ((((110 / traction[0].transformer_ratio)**2/traction[0].nominal_capacity)*((traction[0].short_circuit_voltage_percentage)/100)) * self.h)


        return self.Gn

    def locomotives(self):

        locomotives = self.topology.locomotive

        # print(len(locomotives))

        locomotives_location = []

        locomotives_load = []

        locomotives_num = len(locomotives)

        for i in range(locomotives_num):
            locomotives_location.append(locomotives[i].location)

            locomotives_load.append(locomotives[i].load)
        if self.m == 10 or self.m == 6:

            for i in range(locomotives_num):

                k = int(np.round(locomotives_location[i] / self.delta_length))

                if k < self.n:
                    self.Gn[k * self.m][0] = -1 * locomotives_load[i] + self.Gn[k * self.m][0]

                    self.Gn[k * self.m + 2][0] = locomotives_load[i] + self.Gn[k * self.m + 2][0]  # locomotives_load[i]


        elif self.m == 4:

            for i in range(locomotives_num):

                k = int(np.round(locomotives_location[i] / self.delta_length))

                if k < self.n:
                    self.Gn[k * self.m][0] = -1 * (locomotives_load[i]) + self.Gn[k * self.m][0]

                    self.Gn[k * self.m + 1][0] = locomotives_load[i] + self.Gn[k * self.m + 1][0]


        return self.Gn

    def add_y_AT(self):


        AT_matrix = cc.set_connction_matrix_AT(self.m, 0, 2, 1)

        ATs = self.topology.auto_transformers

        ATs_location = []

        ATs_num = len(ATs)

        for i in range(len(ATs)):
            ATs_location.append(ATs[i].location)

        # print(ATs_location)

        for i in range(ATs_num):

            k = int(np.round(ATs_location[i] / self.delta_length))

            if k < self.n:
                self.y[k, :, :] = self.y[k, :, :] + AT_matrix * (1 / (ATs[i].zs * self.h))  # yg为折算至中点的漏导纳   ATs[i].zs

        return self.y


    def impel(self, impel_index):

        impel = self.topology.locomotive

        impel_num = len(impel)

        # print(impel_num)

        impel_locations = []  # 所有激励的位置
        for i in range(impel_num):
            impel_locations.append(impel[i].location)

        # print(impel_locations)

        k = int(np.round(impel_locations[impel_index] / self.delta_length))

        #         # print(k)

        if k < self.n:
            if self.m == 6:

                self.Gn[k * self.m][0] = -1 + self.Gn[k * self.m][0]

                self.Gn[k * self.m + 2][0] = 1 + self.Gn[k * self.m + 2][0]

            elif self.m == 4:

                self.Gn[k * self.m][0] = -1 + self.Gn[k * self.m][0]

                self.Gn[k * self.m + 1][0] = 1 + self.Gn[k * self.m + 1][0]

            elif self.m == 10:

                self.Gn[k * self.m][0] = -1 + self.Gn[k * self.m][0]

                self.Gn[k * self.m + 2][0] = 1 + self.Gn[k * self.m + 2][0]

        else:

            print("激励位置不在有效区间。")





    def simple_harmonic_plotting(self,line_model=""):#harmonic_num为谐波次数
        U, I = self.solution()                               ###缺省值
        if self.m == 6:
            p = 0
            if line_model == "上行接触线":
                p = 0
            elif line_model == "上行馈线":
                p = 1
            elif line_model == "上行钢轨":
                p = 2
            elif line_model == "下行接触线":
                p = 3
            elif line_model == "下行馈线":
                p = 4
            elif line_model == "下行钢轨":
                p = 5
        elif self.m == 4:
            p = 0
            if line_model == "上行接触线":
                p = 0
            elif line_model == "上行钢轨":
                p = 1
            elif line_model == "下行接触线":
                p = 2
            elif line_model == "下行钢轨":
                p = 3
        elif self.m == 10:
            p = 0
            if line_model == "上行接触线":
                p = 0
            elif line_model == "上行馈线":
                p = 1
            elif line_model == "上行钢轨":
                p = 2
            elif line_model == "上行保护线":
                p = 3
            elif line_model == "上行综合地线":
                p = 4
            elif line_model == "下行接触线":
                p = 5
            elif line_model == "下行馈线":
                p = 6
            elif line_model == "下行钢轨":
                p = 7
            elif line_model == "下行保护线":
                p = 8
            elif line_model == "下行综合地线":
                p = 9
        elif self.m == 4:
            p = 0
            if line_model == "上行接触线":
                p = 0
            elif line_model == "上行钢轨":
                p = 1
            elif line_model == "下行接触线":
                p = 2
            elif line_model == "下行钢轨":
                p = 3
        #绘制某次谐波下的电流特性曲线
        Distances = self.distances
        ####取模值###########################
        simple_U = np.zeros((U.shape[0], U.shape[1]), np.float64)
        simple_I = np.zeros((I.shape[0], I.shape[1]), np.float64)
        for k in range(simple_U.shape[0]):
            for j in range(simple_U.shape[1]):
                simple_U[k][j] = abs(U[k][j])
        for k in range(I.shape[0]):
            for j in range(I.shape[1]):
                simple_I[k][j] = abs(I[k][j])
        plt.plot(Distances[:-1].tolist(), simple_I[p][:].tolist(), color='red')
        if self.h == 1:

            plt.title(u'' + self.topology.lines_system + '工频下'+ line_model +'电流特性曲线', fontproperties=font_set)
            plt.xlabel(u'距离(km)', fontproperties=font_set)
            plt.ylabel(u'电流(A)', fontproperties=font_set)
            plt.show()
            plt.plot(Distances.tolist(), simple_U[p][:].tolist(), color='red')
            plt.title(u'' + self.topology.lines_system + '工频下' + line_model +'电压特性曲线', fontproperties=font_set)
            plt.xlabel(u'距离(km)', fontproperties=font_set)
            plt.ylabel(u'电压(V)', fontproperties=font_set)
            # plt.rcParams['figure.figsize'] = (8.0, 4.0)
            # plt.rcParams['figure.figsize'] = (8.0, 4.0)
            plt.show()

        else:

            plt.title(u'' + self.topology.lines_system + str(self.h) + '次谐波下' + line_model + '电流特性曲线',
                      fontproperties=font_set)
            plt.xlabel(u'距离(km)', fontproperties=font_set)
            plt.ylabel(u'电流i(A)', fontproperties=font_set)
            plt.show()
            plt.plot(Distances.tolist(), simple_U[p][:].tolist(), color='red')
            plt.title(u'' + self.topology.lines_system + str(self.h) + '次谐波下' + line_model + '电压特性曲线',
                      fontproperties=font_set)
            plt.xlabel(u'距离(km)', fontproperties=font_set)
            plt.ylabel(u'电压u(V)', fontproperties=font_set)
            # plt.rcParams['figure.figsize'] = (8.0, 4.0)
            # plt.rcParams['figure.figsize'] = (8.0, 4.0)
            plt.show()




# if __name__ == '__main__':

#     chen = ChainNetwork(h=1, m=6, delta_length=0.5)

#     print("000000000000000000000000000000000000000000000000000000000000000000")

#     chen.reset(1,6,0.5,"calculate_system.db")

#     chen.set_z_y()

#     if chen.m == 6 or chen.m == 10:

#         chen.add_y_AT()

#     YY = chen.add_y_source()

#     chen.add_y()
#     print("AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
#     R = chen.source()
#     #
#     # print(R.shape)
#     print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
#     # chen.impel(0)
#     chen.locomotives()
#     UU, II = chen.solution()
#     print('UU**********************************************************55555555555555555555555555')
#     print(UU)
#     print('II*************************************************************55555555555555555555555555')
#     print(II)

#     if chen.m == 6 or chen.m == 10:

#         chen.simple_harmonic_plotting(line_model="上行接触线")
#         chen.simple_harmonic_plotting(line_model="上行钢轨")
#         chen.simple_harmonic_plotting(line_model="上行馈线")
#         chen.simple_harmonic_plotting(line_model="下行钢轨")
#         chen.simple_harmonic_plotting(line_model="下行接触线")
#         chen.simple_harmonic_plotting(line_model="下行馈线")

#     else:

#         chen.simple_harmonic_plotting(line_model="上行接触线")
#         chen.simple_harmonic_plotting(line_model="上行钢轨")
#         chen.simple_harmonic_plotting(line_model="下行钢轨")
#         chen.simple_harmonic_plotting(line_model="下行接触线")





importing Jupyter notebook from conductor_calculation.ipynb
importing Jupyter notebook from new_topology.ipynb
